# Quarterly comparison of metadata on files with a common template, such as the information template and artwork template

T231952 asks for a Quarterly comparison of metadata on files with a common template, such as the information template and artwork template.
We'll gather both a snapshot of this, as well as look into the number of files having these common templates per quarter. 
<font color= red>_**Note that all metrics calculated in this version of the notebook are different than the previous versions due to the inclusion of December 2019 data that is now available in the Data lake. **_</font>

In [4]:
import pandas as pd
import numpy as np

import datetime as dt

from wmfdata import hive, mariadb

You are using wmfdata 0.1.0 (latest).

You can find the source for `wmfdata` at https://github.com/neilpquinn/wmfdata


### Configuration variables  
**UPDATE 2020-01-17:** Modified Config variables snapshot and end_date to include December 2019 data in all the queries. Refer to [T242816](https://phabricator.wikimedia.org/T242816) 

In [5]:
wmf_snapshot = '2019-12'
start_date = '2019-01-01' # first date of common template use
end_date = '2020-01-01' # last date of common template use

### Number of Files with common templates

We will first get a count of All pages that use Information and Artwork templates on Commonswiki till date.
This can be obtained using the templatelinks table on Mariadb replicas

In [6]:
Count_Template='''
SELECT tl_title as Template_name,
count(distinct tl_from) as Number_of_pages
FROM templatelinks 
WHERE tl_from_namespace = 6 AND tl_namespace = 10 AND tl_title in ("Information","Artwork")  
GROUP BY  tl_title
'''

In [7]:
Total_Count_Template=mariadb.run(Count_Template,'commonswiki')

In [8]:
Total_Count_Template['Number_of_pages'] = Total_Count_Template['Number_of_pages'].astype(int).apply(lambda x: "{:,}".format(x)) 
Total_Count_Template

,Template_name,Number_of_pages
0,Artwork,"2,444,479"
1,Information,"52,468,651"


### Number of Files with common templates from January 2019

We will look at all files created from January 2019, that have Information and Artwork templates used as of today. Note that this is not indicative of whether the common templates were added when the file was first created or added over time after it was created.  
To get this count we will use the Revision table on Mariadb replicas and first timestamp of the file as the date filter. 

In [21]:
File_Count_Information_Template='''
SELECT rev_page, MIN(rev_timestamp) AS first_edit_ts
FROM revision
JOIN templatelinks
ON rev_page = tl_from
WHERE tl_namespace = 10
AND tl_title = "Information"
AND tl_from_namespace = 6
GROUP BY rev_page 
HAVING first_edit_ts >= '20190101000000'
'''

In [22]:
File_Count_Information_Template_2019=mariadb.run(File_Count_Information_Template,'commonswiki')

In [23]:
File_Count_Information_Template_2019.count(0)

rev_page         6392228
first_edit_ts    6392228
dtype: int64

In [2]:
File_Count_Artwork_Template='''
SELECT rev_page, MIN(rev_timestamp) AS first_edit_ts
FROM revision
JOIN templatelinks
ON rev_page = tl_from
WHERE tl_namespace = 10
AND tl_title = "Artwork"
AND tl_from_namespace = 6
GROUP BY rev_page 
HAVING first_edit_ts >= '20190101000000'
'''

In [6]:
File_Count_Artwork_Template_2019=mariadb.run(File_Count_Artwork_Template,'commonswiki')

In [34]:
File_Count_Artwork_Template_2019.count(0)

rev_page         462154
first_edit_ts    462154
dtype: int64

## Quarterly Measurements 
We need a quarterly measurement of the files that have used the Information template and Artwork template. So, we take our query to get the count and use it three times, once for each quarter of 2019.

#### Information Template quarterly comparison

In [24]:
File_Count_Information_Template_2019['first_edit_timestamp'] = pd.to_datetime(File_Count_Information_Template_2019['first_edit_ts'])

Now we can count the number of pages with Information Template for each quarter. Through Q1 (end of March, 2019):



In [25]:
len(File_Count_Information_Template_2019.loc[
    File_Count_Information_Template_2019['first_edit_timestamp'] < dt.datetime(2019, 4, 1, 0, 0, 0)])

1324887

For Q2 (End of June 2019)

In [26]:
Q2_File_Count_Information_Template_2019=len(File_Count_Information_Template_2019.loc[
    (File_Count_Information_Template_2019['first_edit_timestamp'] >= dt.datetime(2019, 4, 1, 0, 0, 0)) & 
    (File_Count_Information_Template_2019['first_edit_timestamp'] < dt.datetime(2019, 7, 1, 0, 0, 0))])

In [27]:
Q2_File_Count_Information_Template_2019

1600128

For Q3 (End of September 2019) 

In [28]:
Q3_File_Count_Information_Template_2019=len(File_Count_Information_Template_2019.loc[
    (File_Count_Information_Template_2019['first_edit_timestamp'] >= dt.datetime(2019, 7, 1, 0, 0, 0)) & (File_Count_Information_Template_2019['first_edit_timestamp'] < dt.datetime(2019, 10, 1, 0, 0, 0))])

In [29]:
Q3_File_Count_Information_Template_2019

1488130

<font color=purple> For Q4 (end of December, 2019):  
***Added for [T242816](https://phabricator.wikimedia.org/T242816) *** </font>


In [30]:
Q4_File_Count_Information_Template_2019=len(File_Count_Information_Template_2019.loc[
    (File_Count_Information_Template_2019['first_edit_timestamp'] >= dt.datetime(2019, 10, 1, 0, 0, 0)) & (File_Count_Information_Template_2019['first_edit_timestamp'] < dt.datetime(2020, 1, 1, 0, 0, 0))])

In [31]:
Q4_File_Count_Information_Template_2019

1558885

#### Artwork Template quarterly comparison

In [8]:
File_Count_Artwork_Template_2019['first_edit_timestamp'] = pd.to_datetime(File_Count_Artwork_Template_2019['first_edit_ts'])

Now we can count the number of pages with Artwork Template for each quarter. Through Q1 (end of March, 2019):


In [36]:
len(File_Count_Artwork_Template_2019.loc[
    File_Count_Artwork_Template_2019['first_edit_timestamp'] < dt.datetime(2019, 4, 1, 0, 0, 0)])

44318

For Q2  (end of June, 2019):

In [37]:
Q2_File_Count_Artwork_Template_2019=len(File_Count_Artwork_Template_2019.loc[
    (File_Count_Artwork_Template_2019['first_edit_timestamp'] >= dt.datetime(2019, 4, 1, 0, 0, 0)) & (File_Count_Artwork_Template_2019['first_edit_timestamp'] < dt.datetime(2019, 7, 1, 0, 0, 0))])

In [38]:
Q2_File_Count_Artwork_Template_2019

64761

For Q3 only (end of September, 2019):



In [39]:
Q3_File_Count_Artwork_Template_2019=len(File_Count_Artwork_Template_2019.loc[
    (File_Count_Artwork_Template_2019['first_edit_timestamp'] >= dt.datetime(2019, 7, 1, 0, 0, 0)) & 
    (File_Count_Artwork_Template_2019['first_edit_timestamp'] < dt.datetime(2019, 10, 1, 0, 0, 0))])

In [40]:
Q3_File_Count_Artwork_Template_2019

270530

<font color=purple> For Q4 (end of December, 2019):  
***Added for [T242816](https://phabricator.wikimedia.org/T242816) ***</font>

In [9]:
Q4_File_Count_Artwork_Template_2019=len(File_Count_Artwork_Template_2019.loc[
    (File_Count_Artwork_Template_2019['first_edit_timestamp'] >= dt.datetime(2019, 10, 1, 0, 0, 0)) & 
    (File_Count_Artwork_Template_2019['first_edit_timestamp'] < dt.datetime(2020, 1, 1, 0, 0, 0))])

In [10]:
Q4_File_Count_Artwork_Template_2019

74138